In [25]:
# Solve a generalized assignment problem between the different tracklets 
# setting similarity weights on the assignments such that we maximize the
# total assignment weigthed objective and match the most likely tracklets
from gurobipy import *
from numpy import *
import os
import fnmatch

In [90]:
print cwd + '/3_obj_less_noisy'

/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/3_obj_less_noisy


In [147]:
import fnmatch
import os
cwd = os.getcwd()
matches = []
for root, dirnames, filenames in os.walk(cwd + '/5_obj_less_noisy'):
    for filename in fnmatch.filter(filenames, 'Pij_2.csv'):
        file_path = os.path.join(root, filename)
        P = loadtxt(open(root + '/' + filename, "rb"), delimiter=",")
#         if root == '/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/3_obj_obsolete/csv_3_1':
        print root
        print P
        numT = shape(P)[0]
        try:
            numC = shape(P)[1]
        except:
            numC = 0;    

        print numC
        T=range(numT)
        C=range(numC)

        # Initialize the model
        m=Model("Assignment")

        # Define the binary variables Xs
        X = []
        for t in T:
            X.append([])
            if numC != 0:
                for c in C:
                    X[t].append(m.addVar(vtype=GRB.BINARY,name="X%d%d"% (t, c)))
            else:
                X[t].append(m.addVar(vtype=GRB.BINARY,name="X%d%d"% (t, 0)))

        # Set the optimization direction
        m.update()
        m.modelSense = GRB.MAXIMIZE


        # Define the assignment constraints
        constraintT = []
        constraintC = []


        if numC != 0:
            # One variable assigned per column
            for t in range(numT):
                constraintT.append(m.addConstr(quicksum(X[t][c] for c in range(numC))
                                               <= 1 ,'constraintT%d' % t))
            # One variable assigned per row
            for c in range(numC):
                constraintT.append(m.addConstr(quicksum(X[t][c] for t in range(numT))
                                               <= 1 ,'constraintC%d' % t))

            # Define objective
            m.setObjective(quicksum(quicksum([X[t][c]*P[t][c] 
                                    for c in range(numC)]) 
                                    for t in range(numT)))
        else:
            # One variable assigned per column
            for t in range(numT):
                constraintT.append(m.addConstr(X[t][0]<= 1 ,'constraintT%d' % t))  

            # One variable assigned per row
            constraintT.append(m.addConstr(quicksum(X[t][0] for t in range(numT))
                                               <= 1 ,'constraintC%d' % t))

            # Define objective                 
            m.setObjective(quicksum(X[t][0]*P[t] for t in range(numT)))



        m.update()

        # Solve the optimization problem
        m.optimize()

        # Print runtime and solution

        print 'runtime is',m.Runtime, 's'
        print m


        if numC != 0:
            Xsol = zeros((numT,numC))
        else:
            Xsol = zeros(numT)
        # Print solution
        if m.status == GRB.Status.OPTIMAL:
            for t in range(numT):
                if numC != 0:
                    for c in range(numC):
                        if X[t][c].x > 0.5:
                            print('%g -> %g' % (t, c))
                            Xsol[t][c] = 1
                else:
                    if X[t][0].x > 0.5:
                        print('0 -> %g' % (t))
                        Xsol[t] = 1
        print Xsol
        name_sol = root + "/Assignment.csv"
        if numC != 0:
            savetxt(name_sol, Xsol, delimiter=",")
        else:
            savetxt(name_sol, Xsol[None], delimiter=",")
        

/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/5_obj_less_noisy/csv_5_59
[[ -9.99990000e+02  -9.99990000e+02  -9.99990000e+02  -9.99990000e+02
   -9.99990000e+02  -9.03330000e-01   1.36330000e+00]
 [ -9.99990000e+02  -9.99990000e+02  -9.99990000e+02  -9.99990000e+02
   -9.99990000e+02   3.63330000e-01  -9.03330000e-01]]
7
Optimize a model with 9 rows, 14 columns and 28 nonzeros
Variable types: 0 continuous, 14 integer (14 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1.72663

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 1.72663 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.726630000000e+00, best bound 1.726630000000e+00, gap 0.0000%
runtime is 0.0196259021759 s
<gurobi.Model MIP instance Assignment: 9 constrs, 14 vars,

Variable types: 0 continuous, 14 integer (14 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1.06

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 1.06 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.060000000000e+00, best bound 1.060000000000e+00, gap 0.0000%
runtime is 0.0184600353241 s
<gurobi.Model MIP instance Assignment: 9 constrs, 14 vars, Parameter changes: LogFile=gurobi.log>
0 -> 6
1 -> 5
[[ 0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  1.  0.]]
/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/5_obj_less_noisy/csv_5_33
[[ -9.99990000e+02  -9.99990000e+02  -9.99990000e+02  -9.99990000e+02
   -9.99990000e+02  -5.70000000e-01   8.63330000e-01  -1.43670000e+00]
 [ -9.99990000e+02  -9.99990000e+02  -9.99990000e+02  -9

7
Optimize a model with 9 rows, 14 columns and 28 nonzeros
Variable types: 0 continuous, 14 integer (14 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.69667

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0.69667 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.966700000000e-01, best bound 6.966700000000e-01, gap 0.0000%
runtime is 0.0188229084015 s
<gurobi.Model MIP instance Assignment: 9 constrs, 14 vars, Parameter changes: LogFile=gurobi.log>
0 -> 6
[[ 0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.]]
/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/5_obj_less_noisy/csv_5_69
[[ -9.99990000e+02  -9.99990000e+02  -9.99990000e+02  -9.99990000e+02
   -9.99990000e+02  -3.45000000e-01   1.03000000e+00]
 [ -9.99990000e

Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.01

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0.01 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e-02, best bound 1.000000000000e-02, gap 0.0000%
runtime is 0.0211510658264 s
<gurobi.Model MIP instance Assignment: 7 constrs, 6 vars, Parameter changes: LogFile=gurobi.log>
0 -> 5
[ 0.  0.  0.  0.  0.  1.]
/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/5_obj_less_noisy/csv_5_64
[[-999.99   -999.99   -999.99   -999.99   -999.99      1.03     -1.3064]
 [-999.99   -999.99   -999.99   -999.99   -999.99   -999.99      1.03  ]]
7
Optimize a model with 9 rows, 14 columns and 28 nonzeros
Variable types: 0 continuous, 14 integ

Variable types: 0 continuous, 21 integer (21 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.72

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0.72 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.200000000000e-01, best bound 7.200000000000e-01, gap 0.0000%
runtime is 0.0193240642548 s
<gurobi.Model MIP instance Assignment: 10 constrs, 21 vars, Parameter changes: LogFile=gurobi.log>
0 -> 5
1 -> 6
[[ 0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.]]
/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/5_obj_less_noisy/csv_5_15
[[ -9.99990000e+02  -9.99990000e+02  -9.99990000e+02  -9.99990000e+02
   -9.99990000e+02  -3.98570000e-01  -3.98570000e-01   1.53000000e+00]
 [ -9.99990000e+02  -9.9

...

Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.01

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0.01 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e-02, best bound 1.000000000000e-02, gap 0.0000%
runtime is 0.0156450271606 s
<gurobi.Model MIP instance Assignment: 7 constrs, 6 vars, Parameter changes: LogFile=gurobi.log>
0 -> 5
[ 0.  0.  0.  0.  0.  1.]
/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/5_obj_less_noisy/csv_5_5
[[ -9.99990000e+02  -9.99990000e+02  -9.99990000e+02  -9.99990000e+02
   -9.99990000e+02   3.30000000e-01   5.30000000e-01]
 [ -9.99990000e+02  -9.99990000e+02  -9.99990000e+02  -9.99990000e+02
   -9.99990000e+02   3.63330000e-01  -3.70000000e-0

Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.01

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0.01 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e-02, best bound 1.000000000000e-02, gap 0.0000%
runtime is 0.0160388946533 s
<gurobi.Model MIP instance Assignment: 7 constrs, 6 vars, Parameter changes: LogFile=gurobi.log>
0 -> 5
[ 0.  0.  0.  0.  0.  1.]
/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/5_obj_less_noisy/csv_5_41
[[ -9.99990000e+02  -9.99990000e+02  -9.99990000e+02  -9.99990000e+02
   -9.99990000e+02  -4.70000000e-01   7.80000000e-01]
 [ -9.99990000e+02  -9.99990000e+02  -9.99990000e+02  -9.99990000e+02
   -9.99990000e+02   1.03000000e+00  -2.20000000e-


Solution count 1: 1.03 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.030000000000e+00, best bound 1.030000000000e+00, gap 0.0000%
runtime is 0.0112979412079 s
<gurobi.Model MIP instance Assignment: 11 constrs, 24 vars, Parameter changes: LogFile=gurobi.log>
0 -> 5
1 -> 7
2 -> 6
[[ 0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.]]
/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/5_obj_less_noisy/csv_5_60
[ -1.00070000e+03  -1.00070000e+03  -1.00070000e+03  -1.00070000e+03
  -1.00070000e+03   1.00000000e-02]
0
Optimize a model with 7 rows, 6 columns and 12 nonzeros
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.01

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 availab

  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1.72667

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 1.72667 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.726670000000e+00, best bound 1.726670000000e+00, gap 0.0000%
runtime is 0.0314939022064 s
<gurobi.Model MIP instance Assignment: 9 constrs, 14 vars, Parameter changes: LogFile=gurobi.log>
0 -> 6
1 -> 5
[[ 0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  1.  0.]]
/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/5_obj_less_noisy/csv_5_61
[[ -9.99990000e+02  -9.99990000e+02  -9.99990000e+02  -9.99990000e+02
   -9.99990000e+02   5.30000000e-01  -1.70000000e-01  -5.70000000e-01]
 [ -9.99990000e+02  -9.99990000e+02  -9.99990000e+02  -9.99990000e+02
   -9.99990000e+02  -2.20000000e-01   3.00000000e-02  -5.

  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 2.27

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 2.27 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.270000000000e+00, best bound 2.270000000000e+00, gap 0.0000%
runtime is 0.0236518383026 s
<gurobi.Model MIP instance Assignment: 9 constrs, 14 vars, Parameter changes: LogFile=gurobi.log>
0 -> 2
1 -> 6
[[ 0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.]]


Optimize a model with 10 rows, 25 columns and 50 nonzeros
Variable types: 0 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: -0 

Optimal solution found (tolerance 1.00e-04)
Best objective -0.000000000000e+00, best bound -0.000000000000e+00, gap 0.0000%


runtime is 0.0195119380951 s
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]
/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/3_obj/csv_3_22
